## 자산 재평가 정보 크롤링

In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd
import numpy as np
import datetime
import time
import os
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import asyncio
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import requests
from urllib import request
from urllib.error import HTTPError
from urllib.error import URLError

today=datetime.datetime.today()
yesterday=today-datetime.timedelta(1)
today=str(pd.to_datetime(today))[:10]
yesterday=str(pd.to_datetime(yesterday))[:10]




files=os.listdir()

In [2]:
url='http://kind.krx.co.kr/disclosure/details.do?method=searchDetailsMain'
path=os.getcwd()
driver=webdriver.Chrome(ChromeDriverManager().install())
driver.get(url)
time.sleep(2)
####공시 100 페이지로 늘리기
#driver.find_element(By.XPATH,'//*[@id="currentPageSize"]').click()
driver.find_element(By.XPATH,'//*[@id="currentPageSize"]/option[4]').click()
driver.find_element(By.XPATH,'//*[@id="main-contents"]/section[2]/div[2]/a').click()


driver.find_element(By.XPATH,'//*[@id="search-btn-dates"]/ul/li[7]/a').click()

driver.find_element(By.XPATH,'//*[@id="searchForm"]/section[1]/div/div[3]/a[1]').click()
time.sleep(5)
driver.find_element(By.XPATH,'//*[@id="dsclsType01"]').click()###수시공시 클릭
driver.find_element(By.XPATH,'//*[@id="dsclsLayer01_35"]').click()###수시공시에서 자산재평가 클릭
driver.find_element(By.XPATH,'//*[@id="searchForm"]/section[1]/div/div[3]/a[1]').click()## 검색 클릭

time.sleep(2)

#########여기까지 필요한 데이터 
num=driver.find_element(By.XPATH,'//*[@id="main-contents"]/section[1]/table/tbody/tr[1]/td[1]').text
num=int(num)
X_path_list=[]

reassess_dict=dict()

company_name_data=driver.find_elements(By.XPATH,f'//*[@id="companysum"]')
for i in range(1,num+1):
    X_path_list.append(f'//*[@id="main-contents"]/section[1]/table/tbody/tr[{i}]/td[4]/a')
    date=driver.find_element(By.XPATH,f'//*[@id="main-contents"]/section[1]/table/tbody/tr[{i}]/td[2]').text
    date=date[:10]
    temp_company_name=company_name_data[i-1].text
    
    
    #announce_date.append(date)
    report_code=driver.find_element(By.XPATH,X_path_list[i-1]).get_attribute("onclick")#### 공시레포트 코드 가져오기
    c_index = report_code.find("(") # 키 값 파싱
    ind_code_r = report_code[(c_index+2):(c_index+2+14)]
    temp_tag=driver.find_element(By.XPATH,f'//*[@id="main-contents"]/section[1]/table/tbody/tr[{i}]/td[4]/a').text
    if ('결과' in temp_tag):
        reassess_dict[ind_code_r]=(date,temp_tag+temp_company_name)


####최종 정정공시만 남기고 나머지 다 지우기
convert_reassess_dict={v:k for k,v in reassess_dict.items()}
flag_list=[ flag for flag in reassess_dict.values() if '정정' in flag[1]]
reassess_dict2=reassess_dict.copy()
for temp_assess in reassess_dict.values():
    for assess_two in flag_list:
        
        if (temp_assess[1] in assess_two[1]) and ('정정' not in temp_assess[1]):
            #print(temp_assess[1])
            reassess_dict2.pop(convert_reassess_dict[temp_assess])




df=pd.DataFrame(columns=['code','name','announce_date','total_ratio'])
tickers=list()
announce_date=list()
name=list()
ratio_list=list()
if len(reassess_dict2.keys()) !=0:
    for report_code in reassess_dict2.keys():
        #if '정정' in reassess_dict[report_code][1]: 
            
        
        
        announce_date.append(reassess_dict2[report_code][0])
        new_url_temp= "https://kind.krx.co.kr/common/disclsviewer.do?method=search&acptno=" + report_code + "&docno=&viewerhost=&viewerport#viewer"
        res=request.urlopen(new_url_temp)
        soup = BeautifulSoup(res, "lxml")
        
        
        nth_options=str(soup.select('#mainDoc')[0]).count('</option>')
        tag1 = soup.select(f'#mainDoc > option:nth-of-type({nth_options})')
        code_name=soup.find('h1',attrs={'class':'ttl type-99 fleft'}).text ###이름 하고 기업 코드 추출
        
        company_name,company_code=code_name.split(' ')
        name.append(company_name)
        company_code=company_code[1:-1]
        company_code='A'+company_code
        tickers.append(company_code)

        x = tag1[0]
        key_text = x.get("value")
        key_report_dir = key_text[0:14]
        #print(key_report_dir)
        new_url=f'https://kind.krx.co.kr/external/{report_code[:4]}/{report_code[4:6]}/{report_code[6:8]}/{report_code[8:14]}/{key_report_dir}/71612.htm'
        
        
        
        
        
        try:
            res=request.urlopen(new_url)
        except HTTPError as e:
            print(e)
            # tag2=soup.select('#mainDoc > option:nth-child(3)')
            # x=tag2[0]
            # key_text = x.get("value")
            # key_report_dir = key_text[0:14]
        try:
            new_url=f'https://kind.krx.co.kr/external/{report_code[:4]}/{report_code[4:6]}/{report_code[6:8]}/{report_code[8:14]}/{key_report_dir}/61612.htm'
            res=request.urlopen(new_url)
        except HTTPError as e:
            print(e)
            
        try:
            new_url=f'https://kind.krx.co.kr/external/{report_code[:4]}/{report_code[4:6]}/{report_code[6:8]}/{report_code[8:14]}/{key_report_dir}/31045.htm'
            res=request.urlopen(new_url)
        except Exception as e:
            print(e)

        soup=BeautifulSoup(res,'lxml')
        j=1
        while True:
            temp=f"#XFormD1_Form0_RepeatTable0 > tbody > tr:nth-child({j}) > td:nth-child(1) > span"
            temp_flag=temps_ratio=soup.select_one(temp).text
            print(temp_flag)
            j+=1
            if '%' in temp_flag: break
        
        ratio_loc=temp.replace('child(1)','child(2)')
        temps_ratio=soup.select(ratio_loc)
            
        """ 
        if new_url[-9:]=='61612.htm':
            temps_ratio=soup.select("#XFormD1_Form0_RepeatTable0 > tbody > tr:nth-child(11) > td:nth-child(2) > span")
            
        else:
            temps_ratio=soup.select('#XFormD1_Form0_RepeatTable0 > tbody > tr:nth-child(10) > td:nth-child(2) > span')"""
            
            
            #XFormD1_Form0_RepeatTable0 > tbody > tr:nth-child(5) > td:nth-child(1)
            #XFormD1_Form0_RepeatTable0 > tbody > tr:nth-child(6) > td:nth-child(1)
            #XFormD1_Form0_RepeatTable0 > tbody > tr:nth-child(11) > td:nth-child(1) > span
            #XFormD1_Form0_RepeatTable0 > tbody > tr:nth-child(11) > td:nth-child(2) > span
            
        ratio=float(temps_ratio[0].text)
        ratio_list.append(ratio)
        
        
data={'code':tickers,'name':name,'announce_date':announce_date,
        'ratio':ratio_list}
new_df=pd.DataFrame(data,columns=['code','name','announce_date','ratio'])
        
new_df=new_df.set_index('code')
new_df.to_csv('asset_assess2.csv')      
driver.close()
        
        
    
    
    
    





C:\Users\exp202207\AppData\Local\Temp\ipykernel_13680\2600555300.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(ChromeDriverManager().install())


HTTP Error 404: Not Found
HTTP Error 404: Not Found
1. 재평가 목적물
2. 재평가 대상
3. 재평가 기준일
4. 재평가 내역(단위:원)
 
합계
자산총액
재평가차액의 자산총액대비 비율(%)
HTTP Error 404: Not Found
HTTP Error 404: Not Found
1. 재평가 목적물
2. 재평가 대상
3. 재평가 기준일
4. 재평가 내역(단위:원)
 
 
 
합계
자산총액
자산총액대비(%)
HTTP Error 404: Not Found
HTTP Error 404: Not Found
1. 재평가 목적물
2. 재평가 대상
3. 재평가 기준일
4. 재평가 내역(단위:원)
 
합계
자산총액
재평가차액의 자산총액대비 비율(%)
HTTP Error 404: Not Found
HTTP Error 404: Not Found
1. 재평가 목적물
2. 재평가 대상
3. 재평가 기준일
4. 재평가 내역(단위:원)
 
 
합계
자산총액
재평가차액의 자산총액대비 비율(%)
HTTP Error 404: Not Found
HTTP Error 404: Not Found
1. 재평가 목적물
2. 재평가 대상
3. 재평가 기준일
4. 재평가 내역(단위:원)
 
합계
자산총액
재평가차액의 자산총액대비 비율(%)
HTTP Error 404: Not Found
HTTP Error 404: Not Found
1. 재평가 목적물
2. 재평가 대상
3. 재평가 기준일
4. 재평가 내역(단위:원)
 
합계
자산총액
재평가차액의 자산총액대비 비율(%)
HTTP Error 404: Not Found
HTTP Error 404: Not Found
1. 재평가 목적물
2. 재평가 대상
3. 재평가 기준일
4. 재평가 내역(단위:원)
 
합계
자산총액
재평가차액의 자산총액대비 비율(%)
HTTP Error 404: Not Found
HTTP Error 404: Not Found
1. 재평가 목적물
2. 재평가 대상
3. 재평가 기준일
4. 재평가 내역(

In [3]:
new_df

,name,announce_date,ratio
code,,,
A286750,나노브릭,2023-03-07,21.47
A011330,유니켐,2023-01-26,7.33
A104460,디와이피엔에프,2023-01-17,16.30
A246960,이노테라피,2023-01-06,11.45
A106240,파인테크닉스,2023-01-06,14.90
A102370,케이옥션,2023-01-03,13.16
A234100,폴라리스세원,2023-01-02,13.17
A000520,삼일제약,2022-12-26,28.58
A306620,네온테크,2022-12-21,27.63


: 

In [4]:
new_df

,name,announce_date,ratio
code,,,
A306620,네온테크,2022-12-21,27.63
A174900,앱클론,2022-11-01,43.63
A317240,TS트릴리온,2022-07-15,12.47
A014990,인디에프,2022-07-11,11.30
A053450,세코닉스,2022-06-30,3.56
A032350,롯데관광개발,2022-06-20,35.58
A043910,자연과환경,2022-02-14,21.91
A096640,멜파스,2022-02-10,20.74
A352940,인바이오,2022-01-20,2.03


In [4]:
#new_df=new_df.set_index('code')
new_df.to_csv('asset_assess2.csv')



In [6]:
new_original=pd.read_csv('asset_assess.csv',index_col=0)

re_assess=pd.concat([new_original,new_df])
re_assess=re_assess.drop_duplicates()
re_assess.to_csv('reassess.csv')

In [91]:
convert_reassess_dict={v:k for k,v in reassess_dict.items()}
flag_list=[ flag for flag in reassess_dict.values() if '정정' in flag[1]]
reassess_dict2=reassess_dict.copy()
for temp_assess in reassess_dict.values():
    for assess_two in flag_list:
        
        if (temp_assess[1] in assess_two[1]) and ('정정' not in temp_assess[1]):
            print(temp_assess[1])
            reassess_dict2.pop(convert_reassess_dict[temp_assess])

자산재평가 결과(자율공시)티에이치엔


In [90]:
assess_two[1]

'[정정]자산재평가 결과(자율공시)티에이치엔'

In [72]:
convert_reassess_dict={v:k for k,v in reassess_dict.items()}
flag_list=[ flag for flag in reassess_dict.values() if '정정' in flag[1]]
reassess_dict2=reassess_dict
for temp_assess in reassess_dict.values():
    for assess_two in flag_list:
        if (temp_assess[1] in assess_two) and ('정정' in temp_assess[1]):
            reassess_dict2.pop(convert_reassess_dict[temp_assess])

In [82]:
reassess_dict2

{'20221101000144': ('2022-11-01', '자산재평가결과(자율공시)앱클론'),
 '20220715000300': ('2022-07-15', '자산재평가결과(자율공시)TS트릴리온'),
 '20220711000234': ('2022-07-11', '자산재평가 결과(자율공시)인디에프'),
 '20220630000501': ('2022-06-30', '자산재평가결과(자율공시)세코닉스'),
 '20220620000452': ('2022-06-20', '자산재평가 결과(자율공시)롯데관광개발'),
 '20220214001128': ('2022-02-14', '자산재평가결과(자율공시)자연과환경'),
 '20220210000241': ('2022-02-10', '자산재평가결과(자율공시)멜파스'),
 '20220120000521': ('2022-01-20', '자산재평가결과(자율공시)인바이오'),
 '20220114000164': ('2022-01-14', '자산재평가결과(자율공시)비디아이'),
 '20220104000167': ('2022-01-04', '자산재평가결과(자율공시)대주전자재료'),
 '20220103000209': ('2022-01-03', '자산재평가결과(자율공시)베셀'),
 '20211230000618': ('2021-12-30', '자산재평가 결과(자율공시)부산주공'),
 '20211229000159': ('2021-12-29', '자산재평가결과(자율공시)이화공영'),
 '20211228000830': ('2021-12-28', '자산재평가결과(자율공시)이즈미디어'),
 '20211224000847': ('2021-12-24', '자산재평가결과(자율공시)동방선기'),
 '20211224000658': ('2021-12-24', '자산재평가결과(자율공시)대성엘텍'),
 '20211216000648': ('2021-12-16', '자산재평가결과(자율공시)위즈코프'),
 '20211208000139': ('2021-12-08', '자산재평가결

In [54]:
data=driver.find_elements(By.XPATH,f'//*[@id="companysum"]')
# /html/body/section[2]/section/article/section[1]/table/tbody/tr[1]/td[3]/a
# /html/body/section[2]/section/article/section[1]/table/tbody/tr[2]/td[3]/a
# //*[@id="companysum"]
print(len(data))
data[0].text

80


'삼일제약'

In [27]:
url='http://kind.krx.co.kr/disclosure/details.do?method=searchDetailsMain'
path=os.getcwd()
driver=webdriver.Chrome(ChromeDriverManager().install())
driver.get(url)
time.sleep(2)
####공시 100 페이지로 늘리기
#driver.find_element(By.XPATH,'//*[@id="currentPageSize"]').click()
driver.find_element(By.XPATH,'//*[@id="currentPageSize"]/option[4]').click()
driver.find_element(By.XPATH,'//*[@id="main-contents"]/section[2]/div[2]/a').click()


driver.find_element(By.XPATH,'//*[@id="search-btn-dates"]/ul/li[7]/a').click()

driver.find_element(By.XPATH,'//*[@id="searchForm"]/section[1]/div/div[3]/a[1]').click()
time.sleep(5)
driver.find_element(By.XPATH,'//*[@id="dsclsType01"]').click()###수시공시 클릭
driver.find_element(By.XPATH,'//*[@id="dsclsLayer01_35"]').click()###수시공시에서 자산재평가 클릭
driver.find_element(By.XPATH,'//*[@id="searchForm"]/section[1]/div/div[3]/a[1]').click()## 검색 클릭

time.sleep(2)

#########여기까지 필요한 데이터 
num=driver.find_element(By.XPATH,'//*[@id="main-contents"]/section[1]/table/tbody/tr[1]/td[1]').text
num=int(num)
X_path_list=[]

reassess_dict=dict()


for i in range(1,num+1):
    X_path_list.append(f'//*[@id="main-contents"]/section[1]/table/tbody/tr[{i}]/td[4]/a')
    date=driver.find_element(By.XPATH,f'//*[@id="main-contents"]/section[1]/table/tbody/tr[{i}]/td[2]').text
    date=date[:10]
    #announce_date.append(date)
    report_code=driver.find_element(By.XPATH,X_path_list[i-1]).get_attribute("onclick")#### 공시레포트 코드 가져오기
    c_index = report_code.find("(") # 키 값 파싱
    ind_code_r = report_code[(c_index+2):(c_index+2+14)]
    temp_tag=driver.find_element(By.XPATH,f'//*[@id="main-contents"]/section[1]/table/tbody/tr[{i}]/td[4]/a').text
    if ('결과' in temp_tag) and ('정정' not in temp_tag):
        reassess_dict[ind_code_r]=date
       

df=pd.DataFrame(columns=['code','name','announce_date','total_ratio'])
tickers=list()
announce_date=list()
name=list()
ratio_list=list()
if len(reassess_dict.keys()) !=0:
    for report_code in reassess_dict.keys():
        announce_date.append(reassess_dict[report_code])
        new_url_temp= "https://kind.krx.co.kr/common/disclsviewer.do?method=search&acptno=" + report_code + "&docno=&viewerhost=&viewerport#viewer"
        res=request.urlopen(new_url_temp)
        soup = BeautifulSoup(res, "lxml")
        
        
        nth_options=str(soup.select('#mainDoc')[0]).count('</option>')
        tag1 = soup.select(f'#mainDoc > option:nth-of-type({nth_options})')
        code_name=soup.find('h1',attrs={'class':'ttl type-99 fleft'}).text ###이름 하고 기업 코드 추출
        
        company_name,company_code=code_name.split(' ')
        name.append(company_name)
        company_code=company_code[1:-1]
        company_code='A'+company_code
        tickers.append(company_code)

        x = tag1[0]
        key_text = x.get("value")
        key_report_dir = key_text[0:14]
        #print(key_report_dir)
        new_url=f'https://kind.krx.co.kr/external/{report_code[:4]}/{report_code[4:6]}/{report_code[6:8]}/{report_code[8:14]}/{key_report_dir}/71612.htm'
        
        
        
        
        
        try:
            res=request.urlopen(new_url)
        except HTTPError as e:
            print(e)
            # tag2=soup.select('#mainDoc > option:nth-child(3)')
            # x=tag2[0]
            # key_text = x.get("value")
            # key_report_dir = key_text[0:14]
        try:
            new_url=f'https://kind.krx.co.kr/external/{report_code[:4]}/{report_code[4:6]}/{report_code[6:8]}/{report_code[8:14]}/{key_report_dir}/61612.htm'
            res=request.urlopen(new_url)
        except HTTPError as e:
            print(e)
            
        try:
            new_url=f'https://kind.krx.co.kr/external/{report_code[:4]}/{report_code[4:6]}/{report_code[6:8]}/{report_code[8:14]}/{key_report_dir}/31045.htm'
            res=request.urlopen(new_url)
        except Exception as e:
            print(e)

        soup=BeautifulSoup(res,'lxml')
        j=1
        while True:
            temp=f"#XFormD1_Form0_RepeatTable0 > tbody > tr:nth-child({j}) > td:nth-child(1) > span"
            temp_flag=temps_ratio=soup.select_one(temp).text
            print(temp_flag)
            j+=1
            if '%' in temp_flag: break
        
        ratio_loc=temp.replace('child(1)','child(2)')
        temps_ratio=soup.select(ratio_loc)
            
        """ 
        if new_url[-9:]=='61612.htm':
            temps_ratio=soup.select("#XFormD1_Form0_RepeatTable0 > tbody > tr:nth-child(11) > td:nth-child(2) > span")
            
        else:
            temps_ratio=soup.select('#XFormD1_Form0_RepeatTable0 > tbody > tr:nth-child(10) > td:nth-child(2) > span')"""
            
            
            #XFormD1_Form0_RepeatTable0 > tbody > tr:nth-child(5) > td:nth-child(1)
            #XFormD1_Form0_RepeatTable0 > tbody > tr:nth-child(6) > td:nth-child(1)
            #XFormD1_Form0_RepeatTable0 > tbody > tr:nth-child(11) > td:nth-child(1) > span
            #XFormD1_Form0_RepeatTable0 > tbody > tr:nth-child(11) > td:nth-child(2) > span
            
        ratio=float(temps_ratio[0].text)
        ratio_list.append(ratio)
        
        
data={'code':tickers,'name':name,'announce_date':announce_date,
        'ratio':ratio_list}
new_df=pd.DataFrame(data,columns=['code','name','announce_date','ratio'])
        
        
        
        
        
    
    
    
    





C:\Users\exp202207\AppData\Local\Temp\ipykernel_3168\1508483018.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(ChromeDriverManager().install())


HTTP Error 404: Not Found
HTTP Error 404: Not Found
1. 재평가 목적물
2. 재평가 대상
3. 재평가 기준일
4. 재평가 내역(단위:원)
 
 
 
합계
자산총액
재평가차액의 자산총액대비 비율(%)
HTTP Error 404: Not Found
HTTP Error 404: Not Found
1. 재평가 목적물
2. 재평가 대상
3. 재평가 기준일
4. 재평가 내역(단위:원)
 
 
 
합계
자산총액
재평가차액의 자산총액대비 비율(%)
HTTP Error 404: Not Found
HTTP Error 404: Not Found
1. 재평가 목적물
2. 재평가 대상
3. 재평가 기준일
4. 재평가 내역(단위:원)
 
 
 
 
합계
자산총액
자산총액대비(%)
HTTP Error 404: Not Found
HTTP Error 404: Not Found
1. 재평가 목적물
2. 재평가 대상
3. 재평가 기준일
4. 재평가 내역(단위:원)
 
합계
자산총액
재평가차액의 자산총액대비 비율(%)
HTTP Error 404: Not Found
HTTP Error 404: Not Found
1. 재평가 목적물
2. 재평가 대상
3. 재평가 기준일
4. 재평가 내역(단위:원)
 
 
합계
자산총액
자산총액대비(%)
HTTP Error 404: Not Found
HTTP Error 404: Not Found
1. 재평가 목적물
2. 재평가 대상
3. 재평가 기준일
4. 재평가 내역(단위:원)
 
합계
자산총액
재평가차액의 자산총액대비 비율(%)
HTTP Error 404: Not Found
HTTP Error 404: Not Found
1. 재평가 목적물
2. 재평가 대상
3. 재평가 기준일
4. 재평가 내역(단위:원)
 
 
합계
자산총액
재평가차액의 자산총액대비 비율(%)
HTTP Error 404: Not Found
HTTP Error 404: Not Found
1. 재평가 목적물
2. 재평가 대상
3. 재평가 기준일
4. 재평가 내

AttributeError: 'NoneType' object has no attribute 'text'

In [37]:
x.text

'[정정]자산재평가 결과(자율공시) (2020.01.14)'

In [36]:
key_report_dir

'20200113000415'

In [62]:
ratio_list.append(4.40)
data={'code':tickers,'name':name,'announce_date':announce_date,
        'ratio':ratio_list}
new_df=pd.DataFrame(data,columns=['code','name','announce_date','ratio'])

In [26]:
new_df=new_df.set_index('code')
new_df.to_csv('new_assess.csv')

In [60]:
new_url_temp

'https://kind.krx.co.kr/common/disclsviewer.do?method=search&acptno=20200108000120&docno=&viewerhost=&viewerport#viewer'

In [11]:
temp

'#XFormD1_Form0_RepeatTable0 > tbody > tr:nth-child(1) > td:nth-child(1) > span'

In [9]:
temps_ratio=soup.select_one(temp).text

AttributeError: 'NoneType' object has no attribute 'text'

In [16]:
tag1

[<option value="20200113000415|Y">[정정]자산재평가 결과(자율공시) (2020.01.14)</option>]

In [16]:
announce_date

['2022-11-01',
 '2022-07-15',
 '2022-07-11',
 '2022-06-30',
 '2022-06-20',
 '2022-02-14',
 '2022-02-10',
 '2022-01-20',
 '2022-01-14',
 '2022-01-04',
 '2022-01-03',
 '2021-12-30',
 '2021-12-29',
 '2021-12-28',
 '2021-12-24',
 '2021-12-24',
 '2021-12-16',
 '2021-12-08',
 '2021-11-18',
 '2021-10-21',
 '2021-07-16',
 '2021-05-11',
 '2021-04-29',
 '2021-04-06',
 '2021-02-17',
 '2021-01-14',
 '2021-01-13',
 '2021-01-12',
 '2021-01-11',
 '2021-01-04',
 '2020-12-23',
 '2020-12-08',
 '2020-12-03',
 '2020-06-25',
 '2020-06-08',
 '2020-03-31',
 '2020-01-20',
 '2020-01-14',
 '2020-01-08']

In [28]:
ratio

12.06

In [20]:
ratio_list

[43.63,
 12.47,
 11.3,
 3.56,
 35.58,
 21.91,
 20.74,
 2.03,
 1.46,
 8.08,
 12.87,
 4.6,
 11.91,
 10.85,
 27.01,
 4.79,
 60.79,
 14.49,
 3.4,
 39.48,
 15.05,
 9.85,
 33.9,
 13.81,
 12.42,
 19.8,
 6.35,
 5.52,
 1.09,
 33.5,
 23.91,
 16.9,
 4.63,
 38.81,
 30.76,
 31.97,
 12.06,
 4.46]

In [29]:
new_url

'https://kind.krx.co.kr/external/2020/01/08/000120/20200113000415/31045.htm'

In [3]:
temp

'#XFormD1_Form0_RepeatTable0 > tbody > tr:nth-child(10) > td:nth-child(1) > span'

In [61]:
temp_flag

'1. 재평가 목적물'

In [31]:
company_name

'티에이치엔'

In [12]:
ratio_list

38

In [54]:
new_url

'https://kind.krx.co.kr/external/2022/06/30/000501/20220523000538/71612.htm'

In [53]:
new_url[-9:]

'71612.htm'

In [25]:
soup.select('#XFormD1_Form0_RepeatTable0 > tbody > tr:nth-child(8) > td:nth-child(2)')

[]

In [41]:
new_url=f'https://kind.krx.co.kr/external/{report_code[:4]}/{report_code[4:6]}/{report_code[6:8]}/{report_code[8:14]}/{key_report_dir}/61612.htm'
res=request.urlopen(new_url)

In [31]:
new_url

'https://kind.krx.co.kr/external/2022/07/11/000234/20220711000493/71612.htm'

In [36]:
new_url

'https://kind.krx.co.kr/external/2022/07/11/000234/20220711000493/71612.htm'